In [61]:
import pandas as pd

#import salling data that is a pickle file 
salling = pd.read_pickle('data/salling_data.pkl')

#import recipe data
recipes = pd.read_csv('data/recipes_data.csv')

In [62]:
#keep the only columns we need
salling = salling[['cluster_id','Store Name', 'Category 1','Cleaned Ingredients', 'Original Price', 'New Price', 'Discount','Percent Discount']]
salling.head()

#print(df['Cleaned Ingredients'].unique())

,cluster_id,Store Name,Category 1,Cleaned Ingredients,Original Price,New Price,Discount,Percent Discount
0,0,Bilka Tilst,Dinner Meals,dinner meal,135.0,99.00,36.00,26.67
1,0,Bilka Tilst,Pork Chops Pork Schnitzels,pork chop pork schnitzel,135.0,63.95,71.05,52.63
2,0,Bilka Tilst,Dinner Meals,dinner meal,59.0,34.95,24.05,40.76
3,0,Bilka Tilst,Dinner Meals,dinner meal,59.0,34.95,24.05,40.76
4,0,Bilka Tilst,Other Pork,pork,89.0,53.95,35.05,39.38


In [63]:
recipes.head(5)

,id,source,link,title,description
0,1,bbc,https://www.bbc.co.uk/food/recipes/chicken_and...,Chicken and tomato pasta,olive oil
1,1,bbc,https://www.bbc.co.uk/food/recipes/chicken_and...,Chicken and tomato pasta,chicken breasts
2,1,bbc,https://www.bbc.co.uk/food/recipes/chicken_and...,Chicken and tomato pasta,butter
3,1,bbc,https://www.bbc.co.uk/food/recipes/chicken_and...,Chicken and tomato pasta,garlic
4,1,bbc,https://www.bbc.co.uk/food/recipes/chicken_and...,Chicken and tomato pasta,rosemary


In [64]:
#how many recipes we have, count unique titles
print(f"there are {recipes['id'].nunique()} unique recipes")

#how many products we have, count unique description_en
print(f"there are {recipes['description'].nunique()} unique ingredients")


there are 1513 unique recipes
there are 2827 unique ingredients


In [65]:
# Merge the dataframes on the 'Category 1' column from df and 'description' column from df2
merged_df = pd.merge(recipes, salling, left_on='description', right_on='Cleaned Ingredients', how='left')

# Display the merged dataframe
merged_df.head(5)

,id,source,link,title,description,cluster_id,Store Name,Category 1,Cleaned Ingredients,Original Price,New Price,Discount,Percent Discount
0,1,bbc,https://www.bbc.co.uk/food/recipes/chicken_and...,Chicken and tomato pasta,olive oil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,bbc,https://www.bbc.co.uk/food/recipes/chicken_and...,Chicken and tomato pasta,chicken breasts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,bbc,https://www.bbc.co.uk/food/recipes/chicken_and...,Chicken and tomato pasta,butter,2.0,Netto Mosede Center,Butter,butter,34.95,22.00,12.95,37.05
3,1,bbc,https://www.bbc.co.uk/food/recipes/chicken_and...,Chicken and tomato pasta,butter,3.0,Netto Højby Odense,Other Butters,butter,21.95,12.00,9.95,45.33
4,1,bbc,https://www.bbc.co.uk/food/recipes/chicken_and...,Chicken and tomato pasta,butter,6.0,Bilka Næstved,Butter,butter,32.95,19.95,13.00,39.45


In [66]:
# Group by recipe id and cluster_id and count the unique matched ingredients
matched_counts = merged_df.dropna(subset=['Cleaned Ingredients']).groupby(['id', 'cluster_id'])['Cleaned Ingredients'].nunique().reset_index()
matched_counts.columns = ['id', 'cluster_id', 'matched_ingredients_count']

# Add a list of the names of the matched ingredients
matched_counts['matched_ingredients'] = matched_counts.apply(
    lambda row: merged_df[(merged_df['id'] == row['id']) & (merged_df['cluster_id'] == row['cluster_id']) & (merged_df['Cleaned Ingredients'].notna())]['Cleaned Ingredients'].unique(), axis=1
)

matched_counts.head()


,id,cluster_id,matched_ingredients_count,matched_ingredients
0,1,2.0,1,[butter]
1,1,3.0,1,[butter]
2,1,6.0,1,[butter]
3,1,8.0,1,[butter]
4,1,10.0,1,[butter]


The only recipes that we are interested in are the ones that have at least one matched ingredient in the Salling data.

The other metric we want to know is how many ingredients are missing from the total of ingredients.

In [67]:
# keep only the recipes id that have more then 1 matched ingredient in that cluster
matched_counts = matched_counts[matched_counts['matched_ingredients_count'] > 1]

print(f"there are {matched_counts['id'].nunique()} recipes that have more than 1 matched ingredient in the same cluster")

there are 171 recipes that have more than 1 matched ingredient in the same cluster


In [75]:
# Group by id and title, and aggregate the descriptions into a list
grouped_recipes = recipes.groupby(['id', 'title'])['description'].apply(list).reset_index()

# Rename the columns
grouped_recipes.columns = ['id', 'title', 'ingredients']

# Display the final DataFrame
grouped_recipes

,id,title,ingredients
0,1,Chicken and tomato pasta,"[olive oil, chicken breasts, butter, garlic, r..."
1,2,Pea-sto pasta,"[peas, garlic, basil, pine nuts, Parmesan, oli..."
2,3,Salmon pasta,"[penne, butter, olive oil, leek, spring onions..."
3,4,"Acqua pazza with hake, fennel and tomatoes","[hake, fennel, garlic, chilli, cherry tomatoes..."
4,5,Adaptable bean soup,"[onions, carrots, olive oil, tomatoes, bay lea..."
...,...,...,...
1508,1509,Quick Tomato and Bean Soup Provencal Recipe,"[olive oil, onion, garlic, minced, diced tomat..."
1509,1510,Red Lentil Chili Recipe,"[grapeseed oil, carrots, chopped, zucchini, cu..."
1510,1511,Beef Miso Ramen Recipe,"[beef sirloin steak, gluten-free soy sauce (ta..."
1511,1512,Cream of Artichoke Soup Recipe,"[whole artichokes, water, chicken stock, dry v..."


In [77]:
# Initialize the unmatched_ingredients and unmatched_ingredients_count columns
matched_counts['unmatched_ingredients'] = [[] for _ in range(len(matched_counts))]
matched_counts['unmatched_ingredients_count'] = 0


for index, row in matched_counts.iterrows():
    # Get the recipe id
    recipe_id = row['id']
    # get list of matched ingredients
    matched_ingredients = row['matched_ingredients']
    # get total list of ingredients
    total_ingredients = grouped_recipes[grouped_recipes['id'] == recipe_id]['ingredients'].values[0]
    # check which ingredients are not matched
    unmatched_ingredients = set(total_ingredients) - set(matched_ingredients)
    # add the unmatched ingredients to the DataFrame
    matched_counts.at[index, 'unmatched_ingredients'] = list(unmatched_ingredients)
    # add the count of unmatched ingredients
    matched_counts.at[index, 'unmatched_ingredients_count'] = len(unmatched_ingredients)

# Display the final DataFrame
matched_counts.head()

,id,cluster_id,matched_ingredients_count,matched_ingredients,unmatched_ingredients,unmatched_ingredients_count
149,13,3.0,2,"[bacon, milk]","[cornflour, eggs, filo pastry, onion, asparagu...",10
150,13,4.0,2,"[bacon, milk]","[cornflour, eggs, filo pastry, onion, asparagu...",10
151,13,5.0,2,"[bacon, milk]","[cornflour, eggs, filo pastry, onion, asparagu...",10
153,13,7.0,2,"[bacon, milk]","[cornflour, eggs, filo pastry, onion, asparagu...",10
154,13,8.0,2,"[bacon, milk]","[cornflour, eggs, filo pastry, onion, asparagu...",10


In [80]:
# print distinct poducts in cluster 20
print(merged_df[merged_df['cluster_id'] == 3]['Cleaned Ingredients'].unique())

['butter' 'yoghurt' 'bacon' 'milk' 'sausage' 'pork' 'chicken' 'yeast'
 'cream cheese' 'apple juice' 'orange juice' 'cheese' 'beef' 'cream'
 'whole milk' 'pork tenderloin' 'rye bread' 'protein' 'pork sausage']


In [86]:
# Create a DataFrame to store the results
results = []

# For each cluster, find the recipe with the most matched ingredients
for cluster_id in merged_df['cluster_id'].unique():
    cluster_df = matched_counts[matched_counts['cluster_id'] == cluster_id]
    if not cluster_df.empty:
        max_matched = cluster_df['matched_ingredients_count'].max()
        recipe_id = cluster_df[cluster_df['matched_ingredients_count'] == max_matched]['id'].values[0]
        recipe_title = merged_df[merged_df['id'] == recipe_id]['title'].iloc[0]
        matched_ingredients = cluster_df[cluster_df['id'] == recipe_id]['matched_ingredients'].values[0]
        unmatched_ingredients = cluster_df[cluster_df['id'] == recipe_id]['unmatched_ingredients'].values[0]
        results.append([cluster_id, recipe_title, matched_ingredients, unmatched_ingredients, max_matched, cluster_df[cluster_df['id'] == recipe_id]['unmatched_ingredients_count'].values[0]])

# Convert the results to a DataFrame
results_df = pd.DataFrame(results, columns=['cluster_id', 'recipe_name', 'matched_ingredients', 'unmatched_ingredients','matched_ingredients_count','unmatched_ingredients_count'])
# for each cluster print the recipe that has the most matched ingredients
results_df = results_df.sort_values(by='cluster_id', ascending=True)
# Display the results DataFrame
results_df.head(5)

,cluster_id,recipe_name,matched_ingredients,unmatched_ingredients,matched_ingredients_count,unmatched_ingredients_count
19,0.0,Dahi papdi chaat,"[potato, yoghurt]","[red onion, tamarind, chaat masala, cumin, pom...",2,12
20,1.0,Gluten-free creamy chicken and mash pie,"[milk, cheese, cream cheese]","[thyme, cornflour, butter, onion, vegetables, ...",3,12
0,2.0,Toffee apple self-saucing pudding,"[butter, milk, apple juice, cream]","[cornflour, eggs, brown sugar, Bramley apples,...",4,11
1,3.0,Dal chicken with chilli paneer and naan,"[yeast, yoghurt, milk, butter]","[soy sauce, fresh coriander, mint, brown sugar...",4,32
42,4.0,Waggoners rice,"[bacon, sausage, chicken]","[beef stock, peppers, onion, olive oil, chilli...",3,10


In [89]:
# print that has more matches 
recipe_with_most_matches = results_df.loc[results_df['matched_ingredients_count'].idxmax()]
print(f"The recipe with the most matches is: {recipe_with_most_matches['recipe_name']} with {recipe_with_most_matches['matched_ingredients_count']} matched ingredients in cluster {recipe_with_most_matches['cluster_id']}")

The recipe with the most matches is: Toffee apple self-saucing pudding with 4 matched ingredients in cluster 2.0


In [90]:
#extract final_counts to a csv file
results_df.to_csv('data/final_counts.csv', index=False)